Were going to imputate several well logs data with model that weh have trained and imported to classified the lithology

First importing all the necessary libraries and dataframe we want to imiputate, and do some light cleaning like replacing the extreme negative values and turn our dataset to numeric values before we imputate the data

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('7_1-1.csv') #Adjust with the csv you want to clean
df.iloc[1:].reset_index(drop=True)
df = df.apply(pd.to_numeric, errors='coerce')
print(df.info())
print(df.head(1))
#df['RXO'].unique()
#df.loc[(df['RXO'] < -200.0) & (df['RXO'] > -999.9999), 'RXO'] = np.nan
#df['RXO'].unique()

There some well logs that have missing feature for the model we create, the empty colum to predict the value also. and we seperate DEPT and filtering dataset with only features used on imputation model

In [ ]:
#df['RSHA'] = np.nan #Adjust Wich feature is missing 
df['RXO'] = np.nan #Adjust Wich feature is missing 
filtered_model = ['GR', 'RHOB', 'NPHI', 'DTC', 'RDEP', 'RXO', 'RSHA', 'FORCE_2020_LITHOFACIES_LITHOLOGY'] #Adjust with the columns you want to impute
df_filtered_model = df[filtered_model]
other_columns = df.drop(columns=filtered_model)
other_columns.info()
df_filtered_model.info()
df_filtered_model_imputed = df_filtered_model.copy()

Importing the model we have trained for imputation. and imputate each missing value.

In [ ]:
import joblib

# Load models
models = {
    'GR': joblib.load('GR_regressor.pkl'),
    'RHOB': joblib.load('RHOB_regressor.pkl'),
    'NPHI': joblib.load('NPHI_regressor.pkl'),
    'DTC': joblib.load('DTC_regressor.pkl'),
    'RDEP': joblib.load('RDEP_regressor.pkl'),
    'RXO': joblib.load('RXO_regressor.pkl'),
    'RSHA': joblib.load('RSHA_regressor.pkl')
}
def impute_column(df, column_name, model):
    missing_mask = df[column_name].isna()
    if missing_mask.sum() > 0:  # Check if there are missing values to impute
        missing_data = df[missing_mask]
        if missing_data.shape[0] > 0:
            features = missing_data.drop(columns=[column_name])
            # Ensure that features do not contain NaNs
            features = features.fillna(features.median())
            predicted_values = model.predict(features)
            df.loc[missing_mask, column_name] = predicted_values
        else:
            print(f"No missing data found for column '{column_name}'.")
    else:
        print(f"No missing values to impute for column '{column_name}'.")
# Impute missing values
for column in models.keys():
    if column in df_filtered_model_imputed.columns:
        impute_column(df_filtered_model_imputed, column, models[column])
# Print the results
print("First few rows of the imputed DataFrame:")
print(df_filtered_model_imputed.head())

print("\nSummary statistics of the DataFrame:")
print(df_filtered_model_imputed.describe())

print("\nCount of missing values in each column:")
print(df_filtered_model_imputed.isna().sum())

# Save the DataFrame to CSV
df_filtered_model_imputed.to_csv('{7_1-1_imputed.csv', index=False)
print("\nDataFrame with imputed values saved to '7_1-1_imputed.csv'.")
df_filtered_model_imputed.info()

In [ ]:
df_filtered_model_imputed.info()

Imputation Result is combined again with the dept value to gave information about the dept for classification

In [ ]:
df_result = pd.concat([other_columns, df_filtered_model_imputed], axis=1)
df_result.info()
df_result.to_csv('{7_1-1_imputed.csv', index=False)